Part 1 - Data preprocessing

Importing the libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
tf.__version__

'2.9.1'

Import dataset

In [4]:
df = pd.read_csv('Churn_Modelling.csv')
X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

In [5]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [6]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

Encoding Categorical Data

Label Encoding the 'Gender' column

Here we are obtaining the index of the gender column, since X is a numpy array and indexing through the column name is not possible

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

One-hot-encoding of 'Geography' column

Here the only thing that changes is the index of teh column upon which we one-hot-encode which is 1 in this case

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

Split dataset into test set and training set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Feature Scaling

It is fundamental to deep learning. So we apply it to everything including dummy variables

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Building the ANN

Initiliazing the ANN

ANN implementation can either be sequential(with a defined input layer, hidden layers and an output ) or a computational graph(where all the neurons are interconnected in a non sequential manner) like in Boltzmann machine.


In our case we initiliase a sequential object upon which our neural network is built

In [12]:
ann = tf.keras.models.Sequential()

Adding input layer and the first hidden layer

the fully connected layer that we add will be built as an object of the dense class

In [13]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

* We must specify the number of neurons we need in our hidden layer. There is no rule of thumb and possibly elbow methods can be used later to arrive at the optimum value

* The activation parameter stands for the activation function that we desire to use. Currently we are using rectifier activation function(relu)

Adding second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding output Layer

In [15]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

* The number of neurons in the output layer depend on the type of output. Currently we have a binary output and hence one neuron would suffice

* If we had 3 possible categories, then would need 3 neurons

* For the output layer, we go for sigmoid activation function since we get the probability along the predictions

### Training the ANN

Compiling the ANN

In [16]:
ann.compile(optimizer ='adam',loss='binary_crossentropy',metrics = ['accuracy'])

Training the ANN on the training set

In [17]:
ann.fit(X_train,y_train,batch_size = 32,epochs=100)

Epoch 1/100
250/250 [==============================] - 4s 2ms/step - loss: 0.5528 - accuracy: 0.7458
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4495 - accuracy: 0.8043
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4275 - accuracy: 0.8146
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4189 - accuracy: 0.8201
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4135 - accuracy: 0.8221
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4091 - accuracy: 0.8211
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4053 - accuracy: 0.8234
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4013 - accuracy: 0.8231
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3985 - accuracy: 0.8255
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3958 - accura

* While in theory we are using stochastic gradient descent, we specify the 'batch_size' for considering those many inputs at a time.

* 'epochs' is used to specify the number of times we need the neural network to train over the input data

For stochastic gradient descent, we opt for the 'adam' optimizer

For non binary classification loss='catergorical_crossentropy' and the activation function in the output layer must be 'softmax'

Predicting result of a single observation

While considering the single variable take care of the following things:
* input be passed as a 2D array
* be careful with the dummy variables
* Scale the input before obtaining the predictions

In [21]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])))

1/1 [==============================] - 0s 32ms/step
[[0.13342997]]


To obtain this in YES/NO form. The threshold will change depending on the problem statement

In [22]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 [==============================] - 0s 46ms/step
[[False]]


Predicting Test Set Results

My_method

In [46]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
output_df = pd.DataFrame(data=y_pred,columns=['predicted_values'])
output_df['actual_values'] = y_test

63/63 [==============================] - 0s 2ms/step


In [39]:
output_df = pd.DataFrame(data=y_pred,columns=['predicted_values'])
output_df['actual_values'] = y_test

In [49]:
def boolean_int(x):
    return int(x)

In [51]:
output_df['predicted_values'] = output_df['predicted_values'].apply(boolean_int)

In [52]:
output_df

,predicted_values,actual_values
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
...,...,...
1995,0,0
1996,1,0
1997,0,0
1998,0,0


Confusion matrix

In [60]:
from sklearn.metrics import confusion_matrix,accuracy_score,plot_confusion_matrix
con = confusion_matrix(y_pred,y_test)

In [61]:
print(con)

[[1348  232]
 [ 247  173]]


In [62]:
print(accuracy_score(y_test,y_pred))

0.7605
